### Criação da tabela 1: contém informações sobre as empresas a serem comparadas

*Downloads e fontes:*
- [Índice ico2 b3](https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-de-sustentabilidade/indice-carbono-eficiente-ico2-composicao-da-carteira.htm): dados das empresas que compõe a carteira do índice ICO2 da B3.
- [Ibovespa - Ações e setores](https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/indice-ibovespa-ibovespa-composicao-da-carteira.htm): dados de todas as empresas que compõe a Ibovespa.

O objetivo de buscar estes dois dados para gerar a primeira tabela é poder criar a coluna binária de particpação do índice e também fazer a comparação setorial.

In [20]:
import pandas as pd
import numpy as np
dados_empresas_ico2 = pd.read_csv('../Downloads/carteira_ico2_set-dez_2023.csv', delimiter=';')
codigo_empresas_ico2 = dados_empresas_ico2['Codigo'].to_list()

dados_empresas_ibovespa = pd.read_csv("../Downloads/ibovespa_acoes_setores.csv", delimiter=';')
dados_empresas_ibovespa = dados_empresas_ibovespa[['Codigo', 'Acao', 'Setor']]
dados_empresas_ibovespa['ico2'] = dados_empresas_ibovespa['Codigo'].apply(lambda x: 1 if x in codigo_empresas_ico2 else 0)
dados_empresas_ibovespa.to_csv("../Database/tabela_1.csv", index=False)
dados_empresas_ibovespa.head()


,Codigo,Acao,Setor,ico2
0,WEGE3,WEG,Bens Indls / Maqs e Equips,1
1,EMBR3,EMBRAER,Bens Indls / Mat Transporte,1
2,AZUL4,AZUL,Bens Indls/Transporte,1
3,CCRO3,CCR SA,Bens Indls/Transporte,1
4,GOLL4,GOL,Bens Indls/Transporte,1


Para selecionar uma quantidade menor de empresas, é feita uma filtragem por setores que compõem e não compõem o índice simultaneamente.

In [21]:
fora_indice = dados_empresas_ibovespa['Setor'].loc[dados_empresas_ibovespa['ico2']== 0 ].unique()
dentro_indice = dados_empresas_ibovespa['Setor'].loc[dados_empresas_ibovespa['ico2']== 1 ].unique()
categorias = []
for categoria in fora_indice:
    if categoria in dentro_indice:
        categorias.append(categoria)

categorias_selecionadas = dados_empresas_ibovespa[dados_empresas_ibovespa['Setor'].isin(categorias)].reset_index()
categorias_selecionadas.head()

,index,Codigo,Acao,Setor,ico2
0,12,ASAI3,ASSAI,Cons N Ciclico / Comercio Distr.,1
1,13,CRFB3,CARREFOUR BR,Cons N Ciclico / Comercio Distr.,0
2,14,PCAR3,P.ACUCAR-CBD,Cons N Ciclico / Comercio Distr.,1
3,18,ARZZ3,AREZZO CO,Consumo Ciclico / Comercio,1
4,19,BHIA3,CASAS BAHIA,Consumo Ciclico / Comercio,1


### Criação da tabela 2

In [22]:
import pandas as pd
import yfinance as yf
import warnings

warnings.filterwarnings('ignore', category=pd.core.common.SettingWithCopyWarning)

acoes_selecionadas = categorias_selecionadas['Codigo'].apply(lambda x: x + '.SA')

data_inicio = "2018-01-01"
data_fim = "2023-01-31"

df_acoes = pd.DataFrame(columns=["Código", "Data", "Preço"])

for codigo in acoes_selecionadas:
    acao = yf.download(codigo, start=data_inicio, end=data_fim)
    acao.reset_index(inplace=True)
    acao = acao[["Date", "Adj Close"]]
    acao.columns = ["Data", "Preço"]
    acao["Código"] = codigo
    df_acoes = pd.concat([df_acoes, acao], ignore_index=True)

df_acoes.head()
df_acoes.to_csv("../Database/tabela_2.csv", index=False)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

### Criação da Tabela 3

In [23]:
import pandas as pd
import yfinance as yf
import warnings

warnings.filterwarnings('ignore', category=pd.core.common.SettingWithCopyWarning)

acoes_ibovespa = categorias_selecionadas['Codigo'].apply(lambda x: x + '.SA')

data_fim = "2022-12-31"

df_acoes = pd.DataFrame(columns=["Código", "Data", "Margem de Lucro", "Receita"])

for codigo in acoes_ibovespa:
    acao = yf.Ticker(codigo)
    info = acao.info
    data = {
        "Código": codigo,
        "Data": data_fim,
        "Margem de Lucro": info.get("profitMargins", None),
        "Receita": info.get("trailingAnnualRevenue", None)
    }
    
    df_acoes = df_acoes.append(data, ignore_index=True)

df_acoes.to_csv("../Database/tabela_3.csv", index=False)
df_acoes.head()


,Código,Data,Margem de Lucro,Receita
0,ASAI3.SA,2022-12-31,0.01503,None
1,CRFB3.SA,2022-12-31,0.00000,None
2,PCAR3.SA,2022-12-31,-0.11210,None
3,ARZZ3.SA,2022-12-31,0.08033,None
4,BHIA3.SA,2022-12-31,-0.03763,None
